# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../project.env")
sys.path.append(os.environ["PYTHONPATH"])

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAIN.architecture.BERT.bert as bert

# Models

## Setup

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import gc

In [4]:
TRAIN_CLASSES = "ten-classes"
num_classes = 10
exploring_batch_size = 1024
base_num_epochs = 2
rate_num_epochs = 3.5
num_class_candidates = list(range(2, num_classes+1, 2))

In [5]:
import json
with open(os.path.join(sup.DATA_ROOT, f"{TRAIN_CLASSES}-subsets.json"), "r") as f:
    loaded = json.load(f)

# Convert keys back to int and lists back to tuples
subsets = {
    int(k): {d: v for d, v in v_dict.items()}
    for k, v_dict in loaded.items()
}
subsets


{2: {'easy': [5, 7], 'average': [2, 4], 'hard': [2, 3]},
 3: {'easy': [0, 3, 5], 'average': [0, 1, 2], 'hard': [0, 1, 4]},
 4: {'easy': [1, 6, 7, 9], 'average': [1, 4, 7, 8], 'hard': [0, 1, 5, 9]},
 5: {'easy': [2, 5, 6, 7, 9],
  'average': [2, 4, 5, 7, 9],
  'hard': [0, 1, 2, 4, 8]},
 6: {'easy': [1, 3, 5, 6, 7, 9],
  'average': [1, 2, 5, 6, 7, 8],
  'hard': [1, 4, 5, 6, 8, 9]},
 7: {'easy': [0, 3, 4, 5, 6, 7, 9],
  'average': [1, 2, 3, 5, 6, 7, 8],
  'hard': [1, 2, 3, 4, 5, 6, 8]},
 8: {'easy': [0, 2, 3, 4, 5, 6, 7, 8],
  'average': [1, 3, 4, 5, 6, 7, 8, 9],
  'hard': [0, 1, 2, 4, 5, 7, 8, 9]},
 9: {'easy': [0, 1, 2, 3, 4, 6, 7, 8, 9],
  'average': [0, 1, 2, 3, 4, 5, 6, 7, 8],
  'hard': [0, 1, 2, 3, 4, 5, 7, 8, 9]},
 10: {'easy': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  'average': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  'hard': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}}

In [6]:
base_data_config = {
  "PH3" : False,
  "reducer" : '',
  "kernel" : '',
  "label_col" : sup.class_numeric_column,
  "batch_size" : exploring_batch_size,
  "class_list" : 'specified'
}

base_train_config = {
  "device" : bert.device,
  "arch" : sup.TRAIN_BERT_CODE,
  "loadable" : bert.DISTILBERT,
  "optimizer" : optim.AdamW,
  "lr" : 1e-5,
  "weight_decay" : 0,
  "loss_fn" : nn.CrossEntropyLoss,
}



## Train

In [7]:
metric_tracker = list()

In [ ]:
for data_unit in [sup.DATA_S_PF, sup.DATA_S_PV]:
  data_config = base_data_config.copy()
  train_config = base_train_config.copy()

  data_config["data_unit"] = data_unit

  for PH2 in [False, True]:
    data_config["PH2"] = PH2
    data_config["n"] = 75 if PH2 else 72

    for n in num_class_candidates:
      s = subsets[n]
      train_config["num_epochs"] = int(base_num_epochs + n * rate_num_epochs)
      for difficulty in ['easy', 'average', 'hard']:
        s_dif = s[difficulty]
        data_config["difficulty"] = difficulty
        data_config["class_numeric_list"] = s_dif
        
        print(data_config)
        print(train_config)
        model = bert.BERT(data_config=data_config, df=None, 
                          train_config=train_config)
        
        model.fit(verbose=True)

        model.test()

        model.full_score()
        print(model.accuracy)
        print(model.top2accuracy)
        print(model.macro_f1)
        print(model.macro_precision)
        print(model.macro_recall)

        metrics = {"data_unit" : data_unit,
                    "PH2" : PH2,
                    "num_classes" : n,
                    "difficulty" : difficulty,
                    "accuracy" : model.accuracy,
                    "top2accuracy" : model.top2accuracy,
                    "macro_f1" : model.macro_f1,
                    "macro_precision" : model.macro_precision,
                    "macro_recall" : model.macro_recall}
        
        model.keep_confusion_matrix()
        model.keep_loss()
        
        metric_tracker.append(metrics.copy())
        
        bert.clean_bert(model)
      
      gc.collect()
      if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    
    gc.collect()
    if torch.backends.mps.is_available():
      torch.mps.empty_cache()
  
  gc.collect()
  if torch.backends.mps.is_available():
    torch.mps.empty_cache()

gc.collect()
if torch.backends.mps.is_available():
  torch.mps.empty_cache()
  



{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spf', 'PH2': False, 'n': 72, 'difficulty': 'easy', 'class_numeric_list': [5, 7]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 9}


0.7660455486542443
1
0.752750911685429
0.8292481203007519
0.7628451380552221
{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spf', 'PH2': False, 'n': 72, 'difficulty': 'average', 'class_numeric_list': [2, 4]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 9}


0.5603112840466926
1
0.4502309556262305
0.7670103092783505
0.5568627450980392
{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spf', 'PH2': False, 'n': 72, 'difficulty': 'hard', 'class_numeric_list': [2, 3]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 9}


0.5892857142857143
1
0.5078658999674515
0.7452119309262166
0.5860944881889764
{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spf', 'PH2': False, 'n': 72, 'difficulty': 'easy', 'class_numeric_list': [1, 6, 7, 9]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 16}


0.7571569595261599
0.926949654491609
0.7562066848532487
0.7633047777343762
0.7599997781955217
{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spf', 'PH2': False, 'n': 72, 'difficulty': 'average', 'class_numeric_list': [1, 4, 7, 8]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 16}


...

0.46745562130177515
0.727810650887574
0.44848481100197346
0.6217668570044251
0.4681547619047619
{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spv', 'PH2': True, 'n': 75, 'difficulty': 'hard', 'class_numeric_list': [0, 1, 2, 4, 5, 7, 8, 9]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 30}


0.47928994082840237
0.6745562130177515
0.45215060118457506
0.4523450080515298
0.4806006493506493
{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spv', 'PH2': True, 'n': 75, 'difficulty': 'easy', 'class_numeric_list': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 37}


0.37264150943396224
0.6415094339622641
0.3351851508983487
0.3512949238050658
0.3762121212121212
{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spv', 'PH2': True, 'n': 75, 'difficulty': 'average', 'class_numeric_list': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 37}


0.42452830188679247
0.6415094339622641
0.41226727274107355
0.4442541659377359
0.4258441558441558
{'PH3': False, 'reducer': '', 'kernel': '', 'label_col': 'class_numeric', 'batch_size': 1024, 'class_list': 'specified', 'data_unit': 'Spv', 'PH2': True, 'n': 75, 'difficulty': 'hard', 'class_numeric_list': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
{'device': device(type='mps'), 'arch': 'BERT', 'loadable': 'distilbert-base-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 37}


0.39622641509433965
0.6698113207547169
0.3811575554392834
0.4111915300007031
0.39692640692640696


# Keep metrics

In [9]:
import pandas as pd

metrics_df = pd.DataFrame(metric_tracker)
metrics_df.sort_values(by='accuracy', ascending=False)

,data_unit,PH2,num_classes,difficulty,accuracy,top2accuracy,macro_f1,macro_precision,macro_recall
15,Spf,True,2,easy,0.979296,1.000000,0.979296,0.979598,0.979532
45,Spv,True,2,easy,0.951220,1.000000,0.951190,0.954545,0.952381
9,Spf,False,8,easy,0.867850,0.950197,0.867756,0.870830,0.868473
12,Spf,False,10,easy,0.854724,0.945669,0.854907,0.859644,0.855402
13,Spf,False,10,average,0.844488,0.941732,0.843554,0.850765,0.845306
14,Spf,False,10,hard,0.839370,0.937795,0.839858,0.844217,0.839995
7,Spf,False,6,average,0.833554,0.935676,0.835053,0.843713,0.832564
6,Spf,False,6,easy,0.832891,0.945623,0.832168,0.839404,0.831504
4,Spf,False,4,average,0.827689,0.972112,0.826138,0.827934,0.830540
10,Spf,False,8,average,0.818182,0.935277,0.819499,0.827587,0.818665


In [10]:
sup.create_dir_if_not_exists(os.path.join(sup.SCORES_ROOT, "specified", sup.TRAIN_BERT_CODE))

metrics_df.to_csv(os.path.join(sup.SCORES_ROOT, "specified", sup.TRAIN_BERT_CODE, f"PREPanalysis{num_classes}.csv"), index=False)